# Machine Learing Development for Sales Forecasting

In [2]:
#@title importing necesary libraries
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pandas as pd
import pingouin as pg
import pickle
#import scikitplot as skplt

from typing import List, Tuple

from pandas.api.types import CategoricalDtype
from statsmodels.stats.contingency_tables import Table2x2
from scipy.stats import randint

from sklearn.compose import make_column_selector,make_column_transformer
from sklearn.preprocessing import StandardScaler, RobustScaler, MinMaxScaler, PowerTransformer, OrdinalEncoder,  OneHotEncoder

from sklearn.experimental import enable_halving_search_cv 
from sklearn.metrics import accuracy_score, log_loss
from sklearn.decomposition import PCA
from sklearn.model_selection import GridSearchCV, train_test_split, cross_val_score,\
                                    learning_curve, validation_curve, RandomizedSearchCV, HalvingRandomSearchCV,\
                                    KFold
                                    
from sklearn.pipeline import Pipeline,make_pipeline

from sklearn.tree import DecisionTreeRegressor
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import GradientBoostingRegressor,RandomForestRegressor

import warnings
warnings.filterwarnings('ignore')

We have performed some feature engineering in the data exploration phase and some feature have therefore become obsolete. We will have to drop those columns. Also we will be using different encoders for the category columns based on its characteristics.

In [3]:
# load the dataset
# load the data (cleaned)
bigmart = pd.read_csv('bigmart_cleaned.csv')

bigmart_copy = bigmart.copy() # create a copy of the data
bigmart_copy.head()

,Item_Identifier,Item_Weight,Item_Fat_Content,Item_Visibility,Item_Type,Item_MRP,Outlet_Identifier,Outlet_Establishment_Year,Outlet_Size,Outlet_Location_Type,Outlet_Type,Item_Outlet_Sales,Outlet_Age,Item_Type_Category,Item_MRP_Category
0,FDA15,9.30,Low Fat,0.016047,Dairy,249.8092,OUT049,1999,Medium,Tier 1,Supermarket Type1,3735.1380,14,Food,Very High
1,DRC01,5.92,Regular,0.019278,Soft Drinks,48.2692,OUT018,2009,Medium,Tier 3,Supermarket Type2,443.4228,4,Drink,Low
2,FDN15,17.50,Low Fat,0.016760,Meat,141.6180,OUT049,1999,Medium,Tier 1,Supermarket Type1,2097.2700,14,Food,High
3,FDX07,19.20,Regular,0.000000,Fruits and Vegetables,182.0950,OUT010,1998,Small,Tier 3,Grocery Store,732.3800,15,Food,High
4,NCD19,8.93,Non-Edible,0.000000,Household,53.8614,OUT013,1987,High,Tier 3,Supermarket Type1,994.7052,26,Non-Consumable,Low


In [ ]:
# create a list of columns to be dropped
columns_dropped = ['Item_Identifier', 'Outlet_Establishment_Year']
# drop irrelevant columns
bigmart_copy.drop[]